In [3]:
from Source import *
import pandas as pd
import os
star_genes = ['CYP2D6', 'CYP2C19', 'HLA-A', 'HLA-B', 'CYP2C9', 'CYP3A4', 'CYP3A5', 'CYP1A2', 'UGT1A4', 'UGT2B15', 'CYP2B6']


director = './validation_panels/panels/PGx-NP_FSA file/'

samples = os.listdir(director)
path_samples = [f'{director}{sample}/' for sample in samples]

# load defintion table
definition = Definition()

# reference size
liz120 = [15, 20, 25, 35, 50, 62, 80, 110, 120]
def call_by_sample(sample, qc_plot =True, intensity_plot =True, update_excel=True):
    sample_folder = f'{director}{sample}/'
    peak_file = f'{sample_folder}{sample}.xlsx'
    print(peak_file)
    s_peak_table = pd.read_excel(peak_file, sheet_name='peak_table')
    # print(s_peak_table.columns)
    target_markers = generate_markers(s_peak_table, new_form=True)
    
    fsa_files = [f'{sample_folder}{fsa}' for fsa in os.listdir(sample_folder) if "fsa" in fsa]

    genotype, support_data = call_from_fsa(sorted(fsa_files), target_markers, liz120, val_plot=False)
    
    s_data = genotype.marker_table(sample)
    

    # genes = list(dict.fromkeys(s_peak_table['gene'].tolist()))
    genes = s_peak_table['gene'].unique()
    # print(genes)
    geno = pd.DataFrame()
    for gene in genes:
        
        if gene in star_genes:
            targets = definition.get_target_marker(gene)
            s_pattern = generate_sample_pattern(s_data, targets['marker_name'])
            result = call_star_allele(s_pattern, targets['pattern'])
            tmp = pd.DataFrame({'sample': sample, 'gene': gene, 
                                'marker': '', 'label': '',
                                'genotype':result}, index=[0])
        else:
            cols = ['sample', 'gene', 'marker', 'label', 'genotype']
            tmp = s_data[s_data['gene'] == gene][cols]
        
        geno = pd.concat([geno, tmp])
    
    print(geno)
     

    if update_excel:
        # book = load_workbook(peak_file, keep_vba=True)
        with pd.ExcelWriter(peak_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            # writer.book = book
            # writer.sheets = {n: book[n] for n in book.sheetnames}
            genotype.allele_table(called_filter=False).to_excel(writer, sheet_name='allele_table', index=False)
            genotype.marker_table().to_excel(writer, sheet_name='marker_table', index=False)
            geno.to_excel(writer, sheet_name='genotype_result', index=False)
            writer.save()
        # book.save(peak_file)
        # book.close()
        
    # QC plots
    if qc_plot:
        qc_figs = genotype.plotting_qc(sample, showfig=False)
        for qc_fig in qc_figs:
            qc_figs.get(qc_fig).savefig(f'{sample_folder}{sample}-{qc_fig}-QC.png', dpi =150)

    # plot raw intensity
    if intensity_plot:
        figs = plot_intensity(genotype, support_data, base_range=(20,85))
        html_file = f'{sample_folder}{sample}_raw_intensity.html'
        if os.path.exists(html_file):
            os.remove(html_file)
        
        with open(html_file, 'a') as f:
            for fig in figs:
                f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
                fig.show()

    return genotype, support_data, s_peak_table

In [4]:
x,y,z = call_by_sample('20220526-630049')

              sample     gene     marker            label genotype
0    20220526-630049   CYP2D6                                     
34   20220526-630049    OPRM1  OPRM1_001  OPRM1 rs1799971       AA
0    20220526-630049    HLA-A                                     
0    20220526-630049    HLA-B                                     
0    20220526-630049   CYP2B6                                     
0    20220526-630049  CYP2C19                                     
0    20220526-630049   CYP2C9                                *1/*1
0    20220526-630049   CYP3A4                                     
0    20220526-630049   CYP3A5                                     
73   20220526-630049    ABCB1  ABCB1_001  ABCB1 rs2032582       AC
76   20220526-630049    ABCB1  ABCB1_002  ABCB1 rs1045642       AG
78   20220526-630049    ANKK1  ANKK1_001  ANKK1 rs1800497       AA
80   20220526-630049     COMT   COMT_001      COMT rs4680       GG
82   20220526-630049     DRD2   DRD2_001   DRD2 rs1799732     

In [3]:
import inspect
os.path.abspath(inspect.getfile(call_by_sample))

'/tmp/ipykernel_271269/3805360137.py'

In [2]:
p.head()

NameError: name 'p' is not defined

In [10]:
g, d, p = call_by_sample('20220526-630049', qc_plot=True)

Index(['package', 'version', 'defined_panel', 'panel', 'gene', 'marker',
       'marker_label', 'is_forward', 'direction', 'base', 'basetype', 'color',
       'min_height', 'min_bin', 'max_bin'],
      dtype='object')
CYP2D6 CYP2D6_001 CYP2D6 *14 S1 nan
G wildtype nan 25.0 35.0 1000
A mutant nan 27.0 36.0 1000
CYP2D6 CYP2D6_002 CYP2D6 *10B S1 nan
C wildtype nan 28.0 38.0 1000
T mutant nan 31.0 37.0 1000
CYP2D6 CYP2D6_003 CYP2D6 *49 S1 nan
T wildtype nan 37.0 44.0 1000
A mutant nan 39.0 45.0 1000
CYP2D6 CYP2D6_004 CYP2D6 *21 S1 nan
G wildtype nan 41.0 45.0 1000
C mutant nan 42.0 49.0 1000
CYP2D6 CYP2D6_005 CYP2D6 *41 S1 nan
G wildtype nan 45.1 50.0 1000
A mutant nan 47.0 50.0 1000
CYP2D6 CYP2D6_006 CYP2D6 *52 S1 nan
G wildtype nan 50.0 53.0 1000
A mutant nan 50.0 55.0 1000
CYP2D6 CYP2D6_007 CYP2D6 *18 S1 nan
G wildtype nan 53.0 57.0 1000
T mutant nan 55.1 59.0 1000
CYP2D6 CYP2D6_008 CYP2D6 *2 S1 nan
C wildtype nan 57.1 62.0 1000
T mutant nan 59.0 63.0 1000
CYP2D6 CYP2D6_009 CYP2D6 *60 S

ValueError: not enough values to unpack (expected 3, got 2)

In [30]:
a = g.allele_table(called_filter=False)

In [34]:
g.marker_table()

,sample,gene,marker,label,panel,direction,genotype,phenotype
0,20889,CYP2D6,CYP2D6_001,CYP2D6 *14,S1,Forward,,
2,20889,CYP2D6,CYP2D6_002,CYP2D6 *10B,S1,Forward,CC,wildtype
4,20889,CYP2D6,CYP2D6_003,CYP2D6 *49,S1,Reverse,,
6,20889,CYP2D6,CYP2D6_004,CYP2D6 *21,S1,Forward,GG,wildtype
8,20889,CYP2D6,CYP2D6_005,CYP2D6 *41,S1,Forward,GG,wildtype
10,20889,CYP2D6,CYP2D6_006,CYP2D6 *52,S1,Reverse,GG,wildtype
12,20889,CYP2D6,CYP2D6_007,CYP2D6 *18,S1,Forward,GG,wildtype
14,20889,CYP2D6,CYP2D6_008,CYP2D6 *2,S1,Reverse,CC,wildtype
16,20889,CYP2D6,CYP2D6_009,CYP2D6 *60,S1,Reverse,GG,wildtype
18,20889,CYP2D6,CYP2D6_010,CYP2D6 *5,S1,Reverse,AA,wildtype


In [11]:
# generate all bin setting for each sample

bin_table = pd.read_excel('./validation_panels/PGx_NP_bin_seting_table_new_form.xlsx')

director = './validation_panels/panels/PGx-NP_FSA file/'

samples = os.listdir(director)

path_samples = [f'{director}{sample}/' for sample in samples]

for sample, path in zip(samples, path_samples):
    name = f'{sample}.xlsx'
    full = f'{path}{name}'
    bin_table.to_excel(full, index=False, sheet_name='peak_table')

In [32]:
def foo(x, y, **d):
    print(x,y, d)
    bar(**d)

def bar(color=20, text=10, **kwargs):
    print(color, text)



foo(1,'ab', color='red')

1 ab {'color': 'red'}
red 10


In [12]:
a = Allele('A', 'a', True, 1,2,390)

1 if not hasattr(a, 'dfsadf') else 0

1